In [89]:
import numpy as np
import pandas as pd
import scipy
from scipy.optimize import minimize
import plotly.express as px
import plotly.graph_objects as go
import datetime

from sklearn import linear_model as lm
from sklearn.feature_selection import r_regression
from sklearn.metrics import r2_score

In [90]:
import warnings
warnings.filterwarnings('ignore')

In [91]:
approval = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vS-FKWVTTFtJT6u56e0bqdfoMcXvDO1DUChsJ3jQAMB2lZk2SMqVfmg7dGjclTYkYWz-Pm5lfcLPjp4/pub?output=csv')
# approval = approval.reset_index()
# approval = approval.rename({'index':'poll_id'}, axis=1)
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,tracking,approve,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp
0,Donald Trump (2nd term),Trade,AtlasIntel,NaN,7/13/2025,7/18/2025,1935.0,A,1.929261,1.59977,NaN,41.0,57.0,-16.0,40.74322,56.313120,-15.569900,https://cdn.atlasintel.org/20e36f18-42bd-4a28-...,7/18/2025,23:00:41 21 Jul 2025
1,Donald Trump (2nd term),Economy,AtlasIntel,NaN,7/13/2025,7/18/2025,1935.0,A,1.929261,1.59977,NaN,40.0,55.0,-15.0,39.56382,54.687570,-15.123750,https://cdn.atlasintel.org/20e36f18-42bd-4a28-...,7/18/2025,23:00:38 21 Jul 2025
2,Donald Trump (2nd term),Immigration,AtlasIntel,NaN,7/13/2025,7/18/2025,1935.0,A,1.929261,1.59977,NaN,45.0,54.0,-9.0,45.44888,51.986820,-6.537941,https://cdn.atlasintel.org/20e36f18-42bd-4a28-...,7/18/2025,23:00:39 21 Jul 2025
3,Donald Trump (2nd term),Adults,AtlasIntel,NaN,7/13/2025,7/18/2025,1935.0,A,1.929261,1.59977,NaN,44.3,54.9,-10.6,43.31635,54.428040,-11.111690,https://cdn.atlasintel.org/20e36f18-42bd-4a28-...,7/18/2025,23:00:32 21 Jul 2025
4,Donald Trump (2nd term),All polls,AtlasIntel,NaN,7/13/2025,7/18/2025,1935.0,A,1.929261,1.59977,NaN,44.3,54.9,-10.6,44.06391,53.635640,-9.571728,https://cdn.atlasintel.org/20e36f18-42bd-4a28-...,7/18/2025,23:00:27 21 Jul 2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1927,Donald Trump (2nd term),Weak,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.749248,0.00000,NaN,15.0,8.0,7.0,12.99547,7.292728,5.702742,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,23:00:36 21 Jul 2025
1928,Donald Trump (2nd term),Immigration,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.749248,0.00000,NaN,46.0,39.0,7.0,47.41845,39.616020,7.802429,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,23:00:39 21 Jul 2025
1929,Donald Trump (2nd term),Adults,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.749248,0.00000,NaN,47.0,41.0,6.0,48.04517,40.836440,7.208729,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,23:00:32 21 Jul 2025
1930,Donald Trump (2nd term),Voters,InsiderAdvantage,NaN,1/20/2025,1/20/2025,800.0,RV,1.093180,0.00000,NaN,56.0,39.0,17.0,53.40312,41.058100,12.345020,https://insideradvantage.com/first-insideradva...,1/20/2025,23:00:30 21 Jul 2025


In [92]:
# approval[approval['pollster']=='YouGov'][approval['subgroup']=='All polls']

In [93]:
approval = approval[~approval['subgroup'].isin(['Weak', 'Strong', 'Economy', 'Cost', 'Immigration', 'Trade', 'Adults', 'All polls'])]

In [94]:
approval.to_csv('trump_approval_polls_RV.csv')

In [95]:
approval.columns.values

array(['president', 'subgroup', 'pollster', 'sponsors', 'startdate',
       'enddate', 'samplesize', 'population', 'weight', 'influence',
       'tracking', 'approve', 'disapprove', 'net', 'adjusted_approve',
       'adjusted_disapprove', 'adjusted_net', 'url', 'createddate',
       'timestamp'], dtype=object)

In [96]:
approval.shape

(341, 20)

In [97]:
# dates = approval['Dates'].str.extract(r'(\d{1,2}\/\d{1,2}) \- (\d{1,2}\/\d{1,2}), (\d{4})')
# dates

In [98]:
# dates['end_date'] = dates[1] + '/' + dates[2]
# dates['end_date'] = pd.to_datetime(dates['end_date'])
# dates

In [99]:
# approval['Dates'] = dates['end_date']
# approval.head()

In [100]:
# samples = approval['Sample'].str.extract(r'[0-9,]{3,5} (\w{1,2})@@(\d{3,4})')
# samples

In [101]:
# approval['population'] = samples[0]
# approval['sample_size'] = samples[1].astype(int)
# approval.head()

In [102]:
ratings = pd.read_csv('pollster_ratings_silver.csv')
ratings

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
0,AtlasIntel,A+@@1,-0.957740,D+0.16@@0.16334188,91,D+
1,Marquette University Law School,A/B@@4,-0.920261,D+0.32@@0.31545374,17,D+
2,The Washington Post,A+@@1,-0.902281,D+0.90@@0.89811021,98,D+
3,Research & Polling Inc.,A@@2,-0.871136,R+0.55@@-0.54944503,44,R+
4,The New York Times/Siena College,A@@2,-0.763947,D+1.41@@1.4121077,116,D+
...,...,...,...,...,...,...
530,Nielson Brothers Polling,C/D@@12,1.946864,D+3.05@@3.0515239,11,D+
531,Ascend Action,C/D@@12,2.282717,D+3.35@@3.3485439,6,D+
532,Jayhawk Consulting Services,C/D@@12,2.353968,D+2.27@@2.2674108,2,D+
533,Fort Hays State University,C/D@@12,2.489170,R+1.10@@-1.0960908,6,R+


In [103]:
ratings['Pollster'].values

array(['AtlasIntel', 'Marquette University Law School',
       'The Washington Post', 'Research & Polling Inc.',
       'The New York Times/Siena College', 'Cygnal', 'SurveyUSA',
       'Siena College', 'Beacon Research/Shaw & Co. Research',
       'Alaska Survey Research', 'OnMessage Inc.', 'TIPP Insights',
       'Emerson College', 'Landmark Communications',
       'Monmouth University', 'Data Orbital',
       'University of North Florida', 'MassINC Polling Group',
       'Marist College', 'Public Policy Institute of California',
       'Public Policy Polling', 'University of California Berkeley',
       'CBS News/The New York Times', 'Christopher Newport University',
       'Mason-Dixon Polling & Strategy', 'Strategic Vision LLC',
       'Suffolk University', 'Stockton University',
       'Field Research Corp. (Field Poll)', 'Quinnipiac University',
       'Los Angeles Times',
       'Hart Research Associates/Public Opinion Strategies',
       'Public Opinion Strategies', 'Susquehan

In [104]:
ratings[ratings['Pollster'].str.contains('North Star')]#.sum()

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
75,North Star Opinion Research,B/C@@8,0.043339,R+0.63@@-0.62697512,13,R+
425,Univision/Latino Decisions/North Star Opinion ...,B/C@@8,0.830409,D+1.06@@1.0564657,7,D+


In [105]:
banned_pollsters = ratings[ratings['Grade'].str.contains('F')]['Pollster'].values
banned_pollsters

array(['Strategic Vision LLC', 'Pharos Research Group', 'Research 2000',
       'Big Data Poll', 'Overtime Politics', 'Rethink Priorities',
       'Blumenthal Research Daily', 'CSP Polling', 'KG Polling',
       'OurProgress (The Progress Campaign)', 'TCJ Research'],
      dtype=object)

In [106]:
approval = approval[~approval['pollster'].isin(banned_pollsters)]
approval.shape

(341, 20)

In [107]:
approval = pd.merge(left=approval, right=ratings[['Pollster', 'Predictive Plus-Minus']], left_on='pollster',
                   right_on='Pollster', how='left')
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus
0,Donald Trump (2nd term),Voters,American Research Group,NaN,7/17/2025,7/20/2025,1006.0,RV,0.889367,0.889367,...,59.0,-21.0,40.17081,57.12141,-16.950600,https://americanresearchgroup.com/economy/,7/20/2025,23:00:30 21 Jul 2025,American Research Group,0.818141
1,Donald Trump (2nd term),Voters,Quinnipiac University,NaN,7/10/2025,7/14/2025,1290.0,RV,1.334559,0.795077,...,54.0,-14.0,42.57624,52.90287,-10.326630,https://poll.qu.edu/poll-release?releaseid=3928,7/14/2025,23:00:30 21 Jul 2025,Quinnipiac University,-0.290484
2,Donald Trump (2nd term),Voters,Morning Consult,NaN,7/18/2025,7/20/2025,2202.0,RV,0.769073,0.769073,...,52.0,-7.0,44.52874,51.81898,-7.290241,https://pro.morningconsult.com/trackers/donald...,7/20/2025,23:00:30 21 Jul 2025,Morning Consult,0.584994
3,Donald Trump (2nd term),Voters,Echelon Insights,NaN,7/10/2025,7/14/2025,1084.0,LV,1.249564,0.745142,...,53.0,-8.0,44.67019,52.74187,-8.071682,https://echelonin.wpenginepowered.com/wp-conte...,7/14/2025,23:00:30 21 Jul 2025,Echelon Insights,-0.216672
4,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,7/14/2025,7/18/2025,1500.0,LV,0.826162,0.676587,...,48.0,2.0,47.31800,49.46945,-2.151451,https://www.rasmussenreports.com/public_conten...,7/18/2025,23:00:30 21 Jul 2025,Rasmussen Reports,0.087015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,Donald Trump (2nd term),Voters,Morning Consult,NaN,1/24/2025,1/26/2025,2302.0,RV,0.795909,0.000000,...,44.0,8.0,51.52874,43.81898,7.709759,https://pro.morningconsult.com/trackers/donald...,1/26/2025,23:00:30 21 Jul 2025,Morning Consult,0.584994
337,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,1/20/2025,1/24/2025,1500.0,LV,0.213379,0.000000,...,43.0,9.0,49.31800,44.46945,4.848549,https://www.rasmussenreports.com/public_conten...,1/24/2025,23:00:30 21 Jul 2025,Rasmussen Reports,0.087015
338,Donald Trump (2nd term),Voters,Echelon Insights,NaN,1/22/2025,1/24/2025,1024.0,LV,1.211361,0.000000,...,43.0,8.0,50.67019,42.74187,7.928318,https://echeloninsights.com/in-the-news/dec-20...,1/24/2025,23:00:30 21 Jul 2025,Echelon Insights,-0.216672
339,Donald Trump (2nd term),Voters,RMG Research,Napolitan News Service,1/20/2025,1/23/2025,3000.0,RV,0.927446,0.000000,...,39.0,18.0,53.93070,41.16710,12.763600,https://x.com/ScottWRasmussen/status/188280455...,1/23/2025,23:00:30 21 Jul 2025,RMG Research,0.278023


In [108]:
approval.shape

(341, 22)

In [109]:
approval['Pollster'].value_counts()

Pollster
Rasmussen Reports                                     126
YouGov                                                 30
Morning Consult                                        26
RMG Research                                           25
J.L. Partners                                          11
Echelon Insights                                        9
Harris Insights & Analytics                             8
Global Strategy Group/GBAO (Navigator Research)         8
Quinnipiac University                                   7
American Research Group                                 6
Emerson College                                         6
McLaughlin & Associates                                 5
John Zogby Strategies                                   5
SSRS                                                    5
Clarity Campaign Labs                                   4
Marquette University Law School                         3
Beacon Research/Shaw & Co. Research                     3
Insid

In [110]:
# approval[~approval['tracking'].isna()]

In [111]:
approval['tracking'].value_counts()

tracking
True    126
Name: count, dtype: int64

In [112]:
# full_approval = approval.copy()

In [113]:
approval = approval[(~approval['samplesize'].isna())]
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus
0,Donald Trump (2nd term),Voters,American Research Group,NaN,7/17/2025,7/20/2025,1006.0,RV,0.889367,0.889367,...,59.0,-21.0,40.17081,57.12141,-16.950600,https://americanresearchgroup.com/economy/,7/20/2025,23:00:30 21 Jul 2025,American Research Group,0.818141
1,Donald Trump (2nd term),Voters,Quinnipiac University,NaN,7/10/2025,7/14/2025,1290.0,RV,1.334559,0.795077,...,54.0,-14.0,42.57624,52.90287,-10.326630,https://poll.qu.edu/poll-release?releaseid=3928,7/14/2025,23:00:30 21 Jul 2025,Quinnipiac University,-0.290484
2,Donald Trump (2nd term),Voters,Morning Consult,NaN,7/18/2025,7/20/2025,2202.0,RV,0.769073,0.769073,...,52.0,-7.0,44.52874,51.81898,-7.290241,https://pro.morningconsult.com/trackers/donald...,7/20/2025,23:00:30 21 Jul 2025,Morning Consult,0.584994
3,Donald Trump (2nd term),Voters,Echelon Insights,NaN,7/10/2025,7/14/2025,1084.0,LV,1.249564,0.745142,...,53.0,-8.0,44.67019,52.74187,-8.071682,https://echelonin.wpenginepowered.com/wp-conte...,7/14/2025,23:00:30 21 Jul 2025,Echelon Insights,-0.216672
4,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,7/14/2025,7/18/2025,1500.0,LV,0.826162,0.676587,...,48.0,2.0,47.31800,49.46945,-2.151451,https://www.rasmussenreports.com/public_conten...,7/18/2025,23:00:30 21 Jul 2025,Rasmussen Reports,0.087015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,Donald Trump (2nd term),Voters,Morning Consult,NaN,1/24/2025,1/26/2025,2302.0,RV,0.795909,0.000000,...,44.0,8.0,51.52874,43.81898,7.709759,https://pro.morningconsult.com/trackers/donald...,1/26/2025,23:00:30 21 Jul 2025,Morning Consult,0.584994
337,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,1/20/2025,1/24/2025,1500.0,LV,0.213379,0.000000,...,43.0,9.0,49.31800,44.46945,4.848549,https://www.rasmussenreports.com/public_conten...,1/24/2025,23:00:30 21 Jul 2025,Rasmussen Reports,0.087015
338,Donald Trump (2nd term),Voters,Echelon Insights,NaN,1/22/2025,1/24/2025,1024.0,LV,1.211361,0.000000,...,43.0,8.0,50.67019,42.74187,7.928318,https://echeloninsights.com/in-the-news/dec-20...,1/24/2025,23:00:30 21 Jul 2025,Echelon Insights,-0.216672
339,Donald Trump (2nd term),Voters,RMG Research,Napolitan News Service,1/20/2025,1/23/2025,3000.0,RV,0.927446,0.000000,...,39.0,18.0,53.93070,41.16710,12.763600,https://x.com/ScottWRasmussen/status/188280455...,1/23/2025,23:00:30 21 Jul 2025,RMG Research,0.278023


In [114]:
auxiliary = pd.read_csv('auxiliary_pollster_rating_stats.csv')
auxiliary

,Pollster,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat
0,AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+
1,The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+
2,Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+
3,The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+
4,Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+
...,...,...,...,...,...,...,...,...
93,Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+
94,Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+
95,Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+
96,McLaughlin & Associates,0.689655,34.5%@@65.517241,R+4.94@@-4.9392128,6.2@@6.17804,0.000000,1,R+


In [115]:
house_eff = auxiliary['House effect'].str.extract(r'(\w)\+(\d\.\d\d)@@*.')
house_eff[1] = house_eff[1].astype(float)
house_eff

,0,1
0,R,2.00
1,D,1.02
2,D,0.63
3,D,0.79
4,R,0.60
...,...,...
93,R,2.44
94,R,0.45
95,R,3.23
96,R,4.94


In [116]:
# R --> -1, D --> 1
house_eff[0] = house_eff[0].replace({'R':-1, 'D':1})
house_eff['house_effect'] = house_eff[0] * house_eff[1]
house_eff

,0,1,house_effect
0,-1,2.00,-2.00
1,1,1.02,1.02
2,1,0.63,0.63
3,1,0.79,0.79
4,-1,0.60,-0.60
...,...,...,...
93,-1,2.44,-2.44
94,-1,0.45,-0.45
95,-1,3.23,-3.23
96,-1,4.94,-4.94


In [117]:
auxiliary['house_effect'] = house_eff['house_effect']
auxiliary = auxiliary.set_index(['Pollster'])
auxiliary

,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat,house_effect
Pollster,,,,,,,,
AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+,-2.00
The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+,1.02
Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+,0.63
The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+,0.79
Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+,-0.60
...,...,...,...,...,...,...,...,...
Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+,-2.44
Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+,-0.45
Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+,-3.23


In [118]:
def apply_house_eff(data, pollster):
    df = data[data['pollster']==pollster]
    try:
        df['approve_adj'] = df['approve'] + (auxiliary.loc[pollster, 'house_effect'] / 2)
        df['disapprove_adj'] = df['disapprove'] - (auxiliary.loc[pollster, 'house_effect'] / 2)
    except KeyError:
        df['approve_adj'] = df['approve']
        df['disapprove_adj'] = df['disapprove']
    return df

In [119]:
pollsters = approval['pollster'].value_counts().index.values
df_adjs = []
for p in pollsters:
    df = apply_house_eff(approval, p)
    df_adjs.append(df)

approval_adj = pd.concat(df_adjs, axis=0)
approval_adj

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus,approve_adj,disapprove_adj
4,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,7/14/2025,7/18/2025,1500.0,LV,0.826162,0.676587,...,47.31800,49.46945,-2.151451,https://www.rasmussenreports.com/public_conten...,7/18/2025,23:00:30 21 Jul 2025,Rasmussen Reports,0.087015,49.360,48.640
11,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,7/11/2025,7/17/2025,1500.0,LV,0.338299,0.236179,...,47.31800,49.46945,-2.151451,https://www.rasmussenreports.com/public_conten...,7/17/2025,23:00:30 21 Jul 2025,Rasmussen Reports,0.087015,49.360,48.640
14,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,7/3/2025,7/10/2025,1500.0,LV,0.380197,0.110978,...,45.31800,51.46945,-6.151451,https://www.rasmussenreports.com/public_conten...,7/10/2025,23:00:30 21 Jul 2025,Rasmussen Reports,0.087015,47.360,50.640
20,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,7/10/2025,7/16/2025,1500.0,LV,0.111466,0.070957,...,46.31800,50.46945,-4.151451,https://www.rasmussenreports.com/public_conten...,7/16/2025,23:00:30 21 Jul 2025,Rasmussen Reports,0.087015,48.360,49.640
22,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,7/9/2025,7/15/2025,1500.0,LV,0.110635,0.063524,...,45.31800,51.46945,-6.151451,https://www.rasmussenreports.com/public_conten...,7/15/2025,23:00:30 21 Jul 2025,Rasmussen Reports,0.087015,47.360,50.640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,Donald Trump (2nd term),Voters,OH Predictive Insights,Center Square,4/15/2025,4/18/2025,2527.0,RV,1.628991,0.000000,...,44.32410,52.58826,-8.264160,https://bloximages.newyork1.vip.townnews.com/t...,4/18/2025,23:00:30 21 Jul 2025,OH Predictive Insights,-0.225885,45.065,51.935
167,Donald Trump (2nd term),Voters,The New York Times/Siena College,NaN,4/21/2025,4/24/2025,913.0,RV,1.420177,0.000000,...,42.72944,53.46981,-10.740370,https://www.nytimes.com/interactive/2025/04/25...,4/24/2025,23:00:30 21 Jul 2025,The New York Times/Siena College,-0.763947,42.395,53.605
161,Donald Trump (2nd term),Voters,NewsNation,NaN,4/23/2025,4/27/2025,1448.0,RV,1.036332,0.000000,...,44.16948,54.81582,-10.646340,https://www.newsnationnow.com/politics/recessi...,4/27/2025,23:00:30 21 Jul 2025,NewsNation,0.614459,44.000,56.000
93,Donald Trump (2nd term),Voters,AP-NORC,NaN,5/1/2025,5/5/2025,1175.0,LV,1.145508,0.000000,...,42.22556,55.33907,-13.113510,https://apnorc.org/wp-content/uploads/2025/05/...,5/5/2025,23:00:30 21 Jul 2025,NaN,NaN,41.000,57.000


In [120]:
approval.shape

(338, 22)

In [121]:
approval = approval_adj.copy()
approval.shape

(338, 24)

In [122]:
approval['poll_id'] = pd.factorize(approval['pollster'] + approval['enddate'] + approval['sponsors'].fillna('none'))[0]
approval.head(7)

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus,approve_adj,disapprove_adj,poll_id
4,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,7/14/2025,7/18/2025,1500.0,LV,0.826162,0.676587,...,49.46945,-2.151451,https://www.rasmussenreports.com/public_conten...,7/18/2025,23:00:30 21 Jul 2025,Rasmussen Reports,0.087015,49.36,48.64,0
11,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,7/11/2025,7/17/2025,1500.0,LV,0.338299,0.236179,...,49.46945,-2.151451,https://www.rasmussenreports.com/public_conten...,7/17/2025,23:00:30 21 Jul 2025,Rasmussen Reports,0.087015,49.36,48.64,1
14,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,7/3/2025,7/10/2025,1500.0,LV,0.380197,0.110978,...,51.46945,-6.151451,https://www.rasmussenreports.com/public_conten...,7/10/2025,23:00:30 21 Jul 2025,Rasmussen Reports,0.087015,47.36,50.64,2
20,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,7/10/2025,7/16/2025,1500.0,LV,0.111466,0.070957,...,50.46945,-4.151451,https://www.rasmussenreports.com/public_conten...,7/16/2025,23:00:30 21 Jul 2025,Rasmussen Reports,0.087015,48.36,49.64,3
22,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,7/9/2025,7/15/2025,1500.0,LV,0.110635,0.063524,...,51.46945,-6.151451,https://www.rasmussenreports.com/public_conten...,7/15/2025,23:00:30 21 Jul 2025,Rasmussen Reports,0.087015,47.36,50.64,4
23,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,7/7/2025,7/11/2025,1500.0,LV,0.152842,0.058551,...,52.46945,-7.151451,https://www.rasmussenreports.com/public_conten...,7/11/2025,23:00:30 21 Jul 2025,Rasmussen Reports,0.087015,47.36,51.64,5
24,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,7/8/2025,7/14/2025,1500.0,LV,0.110236,0.056290,...,52.46945,-8.151451,https://www.rasmussenreports.com/public_conten...,7/14/2025,23:00:30 21 Jul 2025,Rasmussen Reports,0.087015,46.36,51.64,6


In [123]:
approval['population'] = approval['population'].str.strip(' ')

In [124]:
def pipeline(data: pd.DataFrame):
    dup = data[data.duplicated(subset='poll_id', keep=False)]
    ndup = data[~data.duplicated(subset='poll_id', keep=False)]
    # print('dup', dup, '\n ndup', ndup)
    dup_a = dup[dup['population'].isin(['RV'])]
    # print('dup_a', dup_a)
    df = pd.concat([dup_a, ndup], axis=0)
    
    dup = df[df.duplicated(subset='poll_id', keep=False)]
    ndup = df[~df.duplicated(subset='poll_id', keep=False)]
    dup_lv = dup[dup['population'].isin(['LV'])]
    # print('dup_rv', dup_lv, 'ndup', ndup)
    df = pd.concat([dup_lv, ndup], axis=0)
    
    df['end_date'] = pd.to_datetime(df['end_date'])
    
    return df

In [125]:
approval = approval.rename({'enddate':'end_date', 'samplesize':'sample_size', 'Predictive Plus-Minus':'pred_plus_minus'}, axis=1)

In [126]:
approval['population'].value_counts().index.values

array(['RV', 'LV', 'V'], dtype=object)

In [127]:
approval.shape, pipeline(approval).shape

((338, 25), (336, 25))

In [128]:
pipeline(approval)

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id
4,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,7/14/2025,2025-07-18,1500.0,LV,0.826162,0.676587,...,49.46945,-2.151451,https://www.rasmussenreports.com/public_conten...,7/18/2025,23:00:30 21 Jul 2025,Rasmussen Reports,0.087015,49.360,48.640,0
11,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,7/11/2025,2025-07-17,1500.0,LV,0.338299,0.236179,...,49.46945,-2.151451,https://www.rasmussenreports.com/public_conten...,7/17/2025,23:00:30 21 Jul 2025,Rasmussen Reports,0.087015,49.360,48.640,1
14,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,7/3/2025,2025-07-10,1500.0,LV,0.380197,0.110978,...,51.46945,-6.151451,https://www.rasmussenreports.com/public_conten...,7/10/2025,23:00:30 21 Jul 2025,Rasmussen Reports,0.087015,47.360,50.640,2
20,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,7/10/2025,2025-07-16,1500.0,LV,0.111466,0.070957,...,50.46945,-4.151451,https://www.rasmussenreports.com/public_conten...,7/16/2025,23:00:30 21 Jul 2025,Rasmussen Reports,0.087015,48.360,49.640,3
22,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,7/9/2025,2025-07-15,1500.0,LV,0.110635,0.063524,...,51.46945,-6.151451,https://www.rasmussenreports.com/public_conten...,7/15/2025,23:00:30 21 Jul 2025,Rasmussen Reports,0.087015,47.360,50.640,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,Donald Trump (2nd term),Voters,OH Predictive Insights,Center Square,4/15/2025,2025-04-18,2527.0,RV,1.628991,0.000000,...,52.58826,-8.264160,https://bloximages.newyork1.vip.townnews.com/t...,4/18/2025,23:00:30 21 Jul 2025,OH Predictive Insights,-0.225885,45.065,51.935,332
167,Donald Trump (2nd term),Voters,The New York Times/Siena College,NaN,4/21/2025,2025-04-24,913.0,RV,1.420177,0.000000,...,53.46981,-10.740370,https://www.nytimes.com/interactive/2025/04/25...,4/24/2025,23:00:30 21 Jul 2025,The New York Times/Siena College,-0.763947,42.395,53.605,333
161,Donald Trump (2nd term),Voters,NewsNation,NaN,4/23/2025,2025-04-27,1448.0,RV,1.036332,0.000000,...,54.81582,-10.646340,https://www.newsnationnow.com/politics/recessi...,4/27/2025,23:00:30 21 Jul 2025,NewsNation,0.614459,44.000,56.000,334
93,Donald Trump (2nd term),Voters,AP-NORC,NaN,5/1/2025,2025-05-05,1175.0,LV,1.145508,0.000000,...,55.33907,-13.113510,https://apnorc.org/wp-content/uploads/2025/05/...,5/5/2025,23:00:30 21 Jul 2025,NaN,NaN,41.000,57.000,335


In [129]:
# approval[approval['pollster']=='Harris Insights & Analytics']

In [130]:
full_approval = approval.copy()

In [131]:
approval = approval[approval['tracking'].isna()]

In [132]:
# banned_pollsters_aux = ['Rasmussen Reports', 'co/efficient', 'TIPP Insights', 'Civiqs', 'Trafalgar Group', 'OnMessage Inc.']
banned_pollsters_aux = pd.read_csv('banned_pollsters_aux.csv')['banned_pollsters'].values
banned_pollsters_aux

array(['Rasmussen Reports', 'TIPP Insights', 'Trafalgar Group',
       'Trafalgar Group/Insider Advantage'], dtype=object)

In [133]:
approval = approval[~approval['pollster'].isin(banned_pollsters_aux)]
approval.head()

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id
6,Donald Trump (2nd term),Voters,YouGov,Economist,7/11/2025,7/14/2025,1506.0,RV,0.914546,0.576638,...,53.26506,-9.78788,https://d3nkl3psvxxpe9.cloudfront.net/document...,7/14/2025,23:00:30 21 Jul 2025,YouGov,0.047635,43.25,53.75,125
12,Donald Trump (2nd term),Voters,YouGov,Economist,7/4/2025,7/7/2025,1389.0,RV,0.734173,0.200580,...,53.26506,-9.78788,https://d3nkl3psvxxpe9.cloudfront.net/document...,7/7/2025,23:00:30 21 Jul 2025,YouGov,0.047635,43.25,53.75,126
18,Donald Trump (2nd term),Voters,YouGov,Economist,6/27/2025,6/30/2025,1491.0,RV,0.663590,0.072931,...,52.26506,-6.78788,https://d3nkl3psvxxpe9.cloudfront.net/document...,6/30/2025,23:00:30 21 Jul 2025,YouGov,0.047635,45.25,52.75,127
27,Donald Trump (2nd term),Voters,YouGov,Yahoo News,6/26/2025,6/30/2025,1074.0,RV,0.579849,0.049691,...,53.26506,-7.78788,https://www.scribd.com/document/883118567/2025...,6/30/2025,23:00:30 21 Jul 2025,YouGov,0.047635,45.25,53.75,128
33,Donald Trump (2nd term),Voters,YouGov,Economist,6/20/2025,6/23/2025,1455.0,RV,0.688746,0.021816,...,53.26506,-9.78788,https://d3nkl3psvxxpe9.cloudfront.net/document...,6/23/2025,23:00:30 21 Jul 2025,YouGov,0.047635,43.25,53.75,129


In [134]:
approval[approval['pollster']=='Civiqs']# [['approve', 'disapprove', 'approve_adj', 'disapprove_adj']]

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id
115,Donald Trump (2nd term),Voters,Civiqs,Daily Kos,5/17/2025,5/20/2025,1018.0,RV,1.120966,0.0,...,51.04827,-4.226891,https://civiqs.com/documents/Civiqs_DailyKos_b...,5/20/2025,23:00:30 21 Jul 2025,Civiqs,0.158562,47.86,51.14,288
187,Donald Trump (2nd term),Voters,Civiqs,Daily Kos,4/12/2025,4/15/2025,1124.0,RV,1.149724,0.0,...,52.04827,-6.226891,https://civiqs.com/reports/2025/04/17/report-a...,4/15/2025,23:00:30 21 Jul 2025,Civiqs,0.158562,46.86,52.14,289
222,Donald Trump (2nd term),Voters,Civiqs,Daily Kos,2/28/2025,3/3/2025,1031.0,RV,1.124632,0.0,...,51.04827,-3.226891,https://civiqs.com/reports/2025/03/04/report-a...,3/3/2025,23:00:30 21 Jul 2025,Civiqs,0.158562,48.86,51.14,290


In [135]:
approval.columns.values

array(['president', 'subgroup', 'pollster', 'sponsors', 'startdate',
       'end_date', 'sample_size', 'population', 'weight', 'influence',
       'tracking', 'approve', 'disapprove', 'net', 'adjusted_approve',
       'adjusted_disapprove', 'adjusted_net', 'url', 'createddate',
       'timestamp', 'Pollster', 'pred_plus_minus', 'approve_adj',
       'disapprove_adj', 'poll_id'], dtype=object)

In [136]:
approval_polls = approval[['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size', 'population', 'approve', 'disapprove', 'approve_adj', 'disapprove_adj', 'net', 'url', 'pred_plus_minus', 'poll_id']]
approval_polls

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,net,url,pred_plus_minus,poll_id
6,YouGov,Economist,7/11/2025,7/14/2025,1506.0,RV,43.0,54.0,43.250,53.750,-11.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,125
12,YouGov,Economist,7/4/2025,7/7/2025,1389.0,RV,43.0,54.0,43.250,53.750,-11.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,126
18,YouGov,Economist,6/27/2025,6/30/2025,1491.0,RV,45.0,53.0,45.250,52.750,-8.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,127
27,YouGov,Yahoo News,6/26/2025,6/30/2025,1074.0,RV,45.0,54.0,45.250,53.750,-9.0,https://www.scribd.com/document/883118567/2025...,0.047635,128
33,YouGov,Economist,6/20/2025,6/23/2025,1455.0,RV,43.0,54.0,43.250,53.750,-11.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,OH Predictive Insights,Center Square,4/15/2025,4/18/2025,2527.0,RV,44.0,53.0,45.065,51.935,-9.0,https://bloximages.newyork1.vip.townnews.com/t...,-0.225885,332
167,The New York Times/Siena College,NaN,4/21/2025,4/24/2025,913.0,RV,42.0,54.0,42.395,53.605,-12.0,https://www.nytimes.com/interactive/2025/04/25...,-0.763947,333
161,NewsNation,NaN,4/23/2025,4/27/2025,1448.0,RV,44.0,56.0,44.000,56.000,-12.0,https://www.newsnationnow.com/politics/recessi...,0.614459,334
93,AP-NORC,NaN,5/1/2025,5/5/2025,1175.0,LV,41.0,57.0,41.000,57.000,-16.0,https://apnorc.org/wp-content/uploads/2025/05/...,NaN,335


In [137]:
approval_polls.shape

(203, 14)

In [138]:
def avgs_pipeline(data: pd.DataFrame, date: datetime.date):
    state_pivot = pipeline(data)
    # state_pivot['end_date'] = pd.to_datetime(state_pivot['end_date'])
    # state_pivot['pred_plus_minus'] = state_pivot['pred_plus_minus'].map(lambda x: -x)
    state_pivot = state_pivot[state_pivot['end_date'] <= pd.to_datetime(date)]
    state_pivot['population'] = state_pivot['population'].astype(str).map(lambda x: x.strip())
    
    total_num_polls = state_pivot.shape[0]

    # Sample size weights
    total_sample_size = np.sum(state_pivot['sample_size'])
    state_pivot['sample_size_weights'] = (state_pivot['sample_size'].map(lambda x: np.sqrt(min(x, 3000))) / np.sqrt(np.median(state_pivot['sample_size'].map(lambda x: min(x, 2000)))))
    # print(np.median(state_pivot['sample_size'].map(lambda x: min(x, 5000))))
    state_pivot['sample_size_weights'] /= np.sum(state_pivot['sample_size_weights'])
    
    # Time weights
    # Variation of the equation used here: https://polls.votehub.us/
    latest_date = pd.Timestamp(date)
    delta = state_pivot['end_date'].apply(lambda x: (latest_date - x).days)# (latest_date - state_pivot['end_date']).apply(lambda x: x.days)
    linear_weights = (1 - delta/((latest_date - state_pivot['end_date'].min()).days + 1))
    exp_weights = 0.1**(delta/(30))
    state_pivot['time_weights'] =  0 * linear_weights + 1 * exp_weights
    # state_pivot['time_weights'] /= np.sum(state_pivot['time_weights'])
    
    # Quality weights
    rel_quality = state_pivot['pred_plus_minus']
    def quality_weight(rel_qual):
        if np.isnan(rel_qual): # For pollsters without a quality rating from the Silver Bulletin
            return 0.1
        if rel_qual > 0.5: # Utilizing predictive plus-minus from Silver Bulletin's ratings
            return 0.2
        # return np.sqrt(-rel_qual+1)+0.05
        return np.exp(-rel_qual/1.3)
    state_pivot['quality_weights'] = rel_quality.map(quality_weight)
    # state_pivot['quality_weights'] /= np.sum(state_pivot['quality_weights'])
    
    # Population weights
    def population_weight(population):
        if population == 'RV':
            return 0.8
        return 1
    state_pivot['population_weights'] = state_pivot['population'].map(population_weight)
    # state_pivot['population_weights'] /= np.sum(state_pivot['population_weights'])

    # Gather the weights together
    state_pivot['total_weights'] = state_pivot['sample_size_weights'] * state_pivot['time_weights'] * state_pivot['quality_weights'] * state_pivot['population_weights']
    state_pivot['total_weights'] /= np.sum(state_pivot['total_weights']) # Normalization step
    
    return state_pivot

In [139]:
approval_polls['sample_size'].isna().any()

False

In [140]:
avgs_pipeline(approval_polls, datetime.datetime.today())

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,net,url,pred_plus_minus,poll_id,sample_size_weights,time_weights,quality_weights,population_weights,total_weights
6,YouGov,Economist,7/11/2025,2025-07-14,1506.0,RV,43.0,54.0,43.250,53.750,-11.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,125,0.004913,0.584341,0.964021,0.8,0.080308
12,YouGov,Economist,7/4/2025,2025-07-07,1389.0,RV,43.0,54.0,43.250,53.750,-11.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,126,0.004718,0.341455,0.964021,0.8,0.045067
18,YouGov,Economist,6/27/2025,2025-06-30,1491.0,RV,45.0,53.0,45.250,52.750,-8.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,127,0.004888,0.199526,0.964021,0.8,0.027285
27,YouGov,Yahoo News,6/26/2025,2025-06-30,1074.0,RV,45.0,54.0,45.250,53.750,-9.0,https://www.scribd.com/document/883118567/2025...,0.047635,128,0.004149,0.199526,0.964021,0.8,0.023157
33,YouGov,Economist,6/20/2025,2025-06-23,1455.0,RV,43.0,54.0,43.250,53.750,-11.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,129,0.004829,0.116591,0.964021,0.8,0.015750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,OH Predictive Insights,Center Square,4/15/2025,2025-04-18,2527.0,RV,44.0,53.0,45.065,51.935,-9.0,https://bloximages.newyork1.vip.townnews.com/t...,-0.225885,332,0.006364,0.000736,1.189767,0.8,0.000162
167,The New York Times/Siena College,NaN,4/21/2025,2025-04-24,913.0,RV,42.0,54.0,42.395,53.605,-12.0,https://www.nytimes.com/interactive/2025/04/25...,-0.763947,333,0.003825,0.001166,1.799757,0.8,0.000233
161,NewsNation,NaN,4/23/2025,2025-04-27,1448.0,RV,44.0,56.0,44.000,56.000,-12.0,https://www.newsnationnow.com/politics/recessi...,0.614459,334,0.004817,0.001468,0.200000,0.8,0.000041
93,AP-NORC,NaN,5/1/2025,2025-05-05,1175.0,LV,41.0,57.0,41.000,57.000,-16.0,https://apnorc.org/wp-content/uploads/2025/05/...,NaN,335,0.004339,0.002712,0.100000,1.0,0.000043


In [141]:
avgs_pipeline(approval_polls, datetime.datetime.today()).shape

(203, 19)

In [142]:
def approval_over_time(data: pd.DataFrame):
    date_range = pd.date_range(start=datetime.date(2025, 1, 23), end=datetime.datetime.today(), freq='d', inclusive='both')
    yes_averages = []
    no_averages = []
    yes_stdevs = []
    no_stdevs = []
    for date in date_range:
        pipelined_df = avgs_pipeline(data, date)
        yes_avg = np.sum(pipelined_df['approve_adj'] * pipelined_df['total_weights'])
        no_avg = np.sum(pipelined_df['disapprove_adj'] * pipelined_df['total_weights'])
        yes_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['approve_adj'] - yes_avg)**2))
        no_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['disapprove_adj'] - no_avg)**2))
        yes_averages.append(yes_avg)
        no_averages.append(no_avg)
        yes_stdevs.append(yes_std)
        no_stdevs.append(no_std)
    
    return pd.DataFrame({'Date':date_range, 'Approve':yes_averages, 'Disapprove':no_averages}), pd.DataFrame({'Date':date_range, 'Approve':yes_stdevs, 'Disapprove':no_stdevs}), avgs_pipeline(data, datetime.datetime.today())

In [143]:
approval_avg, approval_std, polls_df = approval_over_time(approval_polls)
approval_avg

,Date,Approve,Disapprove
0,2025-01-23,56.560290,39.108753
1,2025-01-24,54.138071,40.803889
2,2025-01-25,54.138071,40.803889
3,2025-01-26,53.978374,41.062509
4,2025-01-27,51.577318,41.773643
...,...,...,...
175,2025-07-17,46.039430,51.025744
176,2025-07-18,46.039430,51.025744
177,2025-07-19,46.039430,51.025744
178,2025-07-20,45.834056,51.227630


In [144]:
polls_df.shape

(203, 19)

In [145]:
polls_df.head(3)

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,net,url,pred_plus_minus,poll_id,sample_size_weights,time_weights,quality_weights,population_weights,total_weights
6,YouGov,Economist,7/11/2025,2025-07-14,1506.0,RV,43.0,54.0,43.25,53.75,-11.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,125,0.004913,0.584341,0.964021,0.8,0.080308
12,YouGov,Economist,7/4/2025,2025-07-07,1389.0,RV,43.0,54.0,43.25,53.75,-11.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,126,0.004718,0.341455,0.964021,0.8,0.045067
18,YouGov,Economist,6/27/2025,2025-06-30,1491.0,RV,45.0,53.0,45.25,52.75,-8.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,127,0.004888,0.199526,0.964021,0.8,0.027285


In [146]:
polls_df['net_adj'] = polls_df['approve_adj'] - polls_df['disapprove_adj']

In [147]:
display_df = polls_df[['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size', 'population', 'approve', 'disapprove', 'net',
                      'total_weights', 'url']]
display_df['sponsors'] = '^Sponsor: ' + display_df['sponsors'] + '^'
display_df['sponsors'] = display_df['sponsors'].fillna('')
display_df = display_df.sort_values(['end_date'], ascending=False)
display_df = display_df.rename({'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 'total_weights':'Weight', 'startdate':'Start Date', 'end_date':'End Date', 'url':'URL'}, axis='columns')
display_df['Sample'] = display_df['sample_size'].astype(int).astype(str) + ' ' + display_df['population']
# display_df['end_date'] = display_df['end_date'].map(lambda x: datetime.datetime.strptime(datetime.datetime.fromtimestamp(x), '%m/%d/%Y').strftime('%Y-%m-%d'))
# display_df['Date'] = pd.to_datetime(display_df['startdate']).astype(str) + ' - ' + display_df['end_date'].astype(str)
display_df['Start Date'] = pd.to_datetime(display_df['Start Date']).astype(str)
display_df['End Date'] = display_df['End Date'].astype(str)
display_df = display_df.reset_index()
# display_df = display_df.drop(['sample_size', 'population'], axis='columns')
display_df['Pollster'] = '<a href="' + display_df['URL'] + '" style="color:black; text-decoration:underline;"><u>' + display_df['Pollster'] + '</u></a>' + display_df['sponsors']
# display_df['Pollster'] = '[' + display_df['Pollster'] + '](' + display_df['URL'] + ')'
display_df = display_df[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net']]
# display_df = display_df.set_index(['Pollster'])
display_df

,Pollster,Start Date,End Date,Sample,Weight,Approve,Disapprove,Net
0,"<a href=""https://pro.morningconsult.com/tracke...",2025-07-18,2025-07-20,2202 RV,3.192983e-02,45.0,52.0,-7.0
1,"<a href=""https://americanresearchgroup.com/eco...",2025-07-17,2025-07-20,1006 RV,2.158176e-02,38.0,59.0,-21.0
2,"<a href=""https://napolitannews.org/jobapproval...",2025-07-09,2025-07-16,3000 RV,1.106892e-01,52.0,48.0,4.0
3,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-07-11,2025-07-14,1506 RV,8.030764e-02,43.0,54.0,-11.0
4,"<a href=""https://poll.qu.edu/poll-release?rele...",2025-07-10,2025-07-14,1290 RV,9.640411e-02,40.0,54.0,-14.0
...,...,...,...,...,...,...,...,...
198,"<a href=""https://poll.qu.edu/poll-release?rele...",2025-01-23,2025-01-27,1019 RV,2.152226e-07,46.0,43.0,3.0
199,"<a href=""https://pro.morningconsult.com/tracke...",2025-01-24,2025-01-26,2302 RV,4.791895e-08,52.0,44.0,8.0
200,"<a href=""https://echeloninsights.com/in-the-ne...",2025-01-22,2025-01-24,1024 LV,2.023961e-07,51.0,43.0,8.0
201,"<a href=""https://x.com/ScottWRasmussen/status/...",2025-01-20,2025-01-23,3000 RV,1.754306e-07,57.0,39.0,18.0


In [148]:
display_df.to_csv('polls_display_table_RV.csv')

In [149]:
approval_std

,Date,Approve,Disapprove
0,2025-01-23,0.717600,0.247043
1,2025-01-24,2.809220,1.938337
2,2025-01-25,2.809220,1.938337
3,2025-01-26,2.720505,2.013544
4,2025-01-27,4.107972,2.011968
...,...,...,...
175,2025-07-17,3.826841,2.889251
176,2025-07-18,3.826841,2.889251
177,2025-07-19,3.826841,2.889251
178,2025-07-20,3.928394,3.073626


In [150]:
approval_avg['Net'] = approval_avg['Approve'] - approval_avg['Disapprove']
approval_avg

,Date,Approve,Disapprove,Net
0,2025-01-23,56.560290,39.108753,17.451538
1,2025-01-24,54.138071,40.803889,13.334182
2,2025-01-25,54.138071,40.803889,13.334182
3,2025-01-26,53.978374,41.062509,12.915866
4,2025-01-27,51.577318,41.773643,9.803676
...,...,...,...,...
175,2025-07-17,46.039430,51.025744,-4.986314
176,2025-07-18,46.039430,51.025744,-4.986314
177,2025-07-19,46.039430,51.025744,-4.986314
178,2025-07-20,45.834056,51.227630,-5.393574


In [151]:
approval_trendline = pd.merge(left=approval_avg, right=approval_std, left_on='Date', right_on='Date', how='inner')
approval_trendline['approve_CI_upper'] = approval_trendline['Approve_x'] + 1.96*approval_trendline['Approve_y']
approval_trendline['approve_CI_lower'] = approval_trendline['Approve_x'] - 1.96*approval_trendline['Approve_y']
approval_trendline['disapprove_CI_upper'] = approval_trendline['Disapprove_x'] + 1.96*approval_trendline['Disapprove_y']
approval_trendline['disapprove_CI_lower'] = approval_trendline['Disapprove_x'] - 1.96*approval_trendline['Disapprove_y']
approval_trendline = approval_trendline.rename({
    'Approve_x':'approve_mean',
    'Disapprove_x':'disapprove_mean',
}, axis='columns').drop(['Approve_y', 'Disapprove_y', 'Net'], axis='columns')
approval_trendline

,Date,approve_mean,disapprove_mean,approve_CI_upper,approve_CI_lower,disapprove_CI_upper,disapprove_CI_lower
0,2025-01-23,56.560290,39.108753,57.966787,55.153793,39.592956,38.624549
1,2025-01-24,54.138071,40.803889,59.644143,48.631999,44.603031,37.004748
2,2025-01-25,54.138071,40.803889,59.644143,48.631999,44.603031,37.004748
3,2025-01-26,53.978374,41.062509,59.310565,48.646184,45.009055,37.115962
4,2025-01-27,51.577318,41.773643,59.628943,43.525694,45.717101,37.830185
...,...,...,...,...,...,...,...
175,2025-07-17,46.039430,51.025744,53.540038,38.538822,56.688677,45.362811
176,2025-07-18,46.039430,51.025744,53.540038,38.538822,56.688677,45.362811
177,2025-07-19,46.039430,51.025744,53.540038,38.538822,56.688677,45.362811
178,2025-07-20,45.834056,51.227630,53.533708,38.134404,57.251937,45.203323


In [152]:
# Export approval ratings
approval_trendline.to_csv('approval_stats_RV.csv')

In [153]:
polls_df.shape

(203, 20)

In [154]:
line = px.line(approval_avg, x='Date', y=['Approve', 'Disapprove'])
# scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['approve', 'disapprove'])
line.update_traces(hovertemplate=None)
line.update_layout(hovermode='x unified')
# scatter.update_traces(hovertemplate=None, hoverinfo='skip')
fig_approve_CI = go.Figure([
    go.Scatter(
        name='Approve CI Upper Bound',
        x = approval_avg['Date'],
        y = approval_avg['Approve'] + 1.96*approval_std['Approve'],
        mode='lines',
        marker=dict(color='#8be8f2'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    go.Scatter(
        name='Approve CI Lower Bound',
        x = approval_avg['Date'],
        y = approval_avg['Approve'] - 1.96*approval_std['Approve'],
        mode='lines',
        marker=dict(color='#8be8f2'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),  
])
fig_disapprove_CI = go.Figure([
    go.Scatter(
        name='Disapprove CI Upper Bound',
        x = approval_avg['Date'],
        y = approval_avg['Disapprove'] + 1.96*approval_std['Disapprove'],
        mode='lines',
        marker=dict(color='#a3cbec'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    go.Scatter(
        name='Disapprove CI Lower Bound',
        x = approval_avg['Date'],
        y = approval_avg['Disapprove'] - 1.96*approval_std['Disapprove'],
        mode='lines',
        marker=dict(color='#a3cbec'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),  
])

fig = go.Figure(data=line.data + fig_approve_CI.data + fig_disapprove_CI.data)
fig.update_layout(
    title='President Trump Approval Rating',
    xaxis_title='Date',
    yaxis_title='%',
    legend_title='Legend',
    hovermode='x unified'
)
fig.show()

In [155]:
line = px.line(approval_avg, x='Date', y=['Net'])
scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['net'])
                     #trendline='lowess', trendline_options=dict(frac=0.2))
fig = go.Figure(data=line.data + scatter.data)
fig.add_hline(y=0, line_dash='dash')
fig.show()

In [156]:
# scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['net', 'net_adj'])
# fig = go.Figure(data=scatter.data)
# fig.add_hline(y=0, line_dash='dash')
# fig.show()

In [157]:
def net_col(df: pd.DataFrame, min_date: datetime.date):
#     df_copy = df.set_index(['Date'])
#     return df_copy['approve_mean'] - df_copy['disapprove_mean']
    date_range = pd.date_range(start=min_date, end=datetime.datetime.today(), freq='d', inclusive='both')
    data = df.copy()
    data['net_adj'] = data['approve_adj'] - data['disapprove_adj']
    net_averages = []
    net_stdevs = []
    issue_cols = {'Date': date_range, 'net':[], 'net_std':[]}
    for date in date_range:
    #         try:
            pipelined_df = avgs_pipeline(data, date)
    #         except TypeError:
    #             print('TypeError')
    #             print(date)
            net_avg = np.sum(pipelined_df['net_adj'] * pipelined_df['total_weights'])
            net_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['net_adj'] - net_avg)**2))

            net_averages.append(net_avg)
            net_stdevs.append(net_std)

    issue_cols['net'].extend(net_averages)
    issue_cols[f'net_std'].extend(net_stdevs)
        
    net_averages, net_stdevs = [], []
        
    return pd.DataFrame(issue_cols)

In [158]:
net_approval = net_col(approval, datetime.date(2025, 1, 23))
net_approval

,Date,net,net_std
0,2025-01-23,17.451538,0.964643
1,2025-01-24,13.334182,4.742140
2,2025-01-25,13.334182,4.742140
3,2025-01-26,12.915866,4.699150
4,2025-01-27,9.803676,5.827191
...,...,...,...
175,2025-07-17,-4.986314,6.426807
176,2025-07-18,-4.986314,6.426807
177,2025-07-19,-4.986314,6.426807
178,2025-07-20,-5.393574,6.732612


In [159]:
net_approval['net_upper_CI'] = net_approval['net'] + 1.96*net_approval['net_std']
net_approval['net_lower_CI'] = net_approval['net'] - 1.96*net_approval['net_std']
net_approval

,Date,net,net_std,net_upper_CI,net_lower_CI
0,2025-01-23,17.451538,0.964643,19.342238,15.560837
1,2025-01-24,13.334182,4.742140,22.628777,4.039587
2,2025-01-25,13.334182,4.742140,22.628777,4.039587
3,2025-01-26,12.915866,4.699150,22.126199,3.705532
4,2025-01-27,9.803676,5.827191,21.224970,-1.617619
...,...,...,...,...,...
175,2025-07-17,-4.986314,6.426807,7.610227,-17.582855
176,2025-07-18,-4.986314,6.426807,7.610227,-17.582855
177,2025-07-19,-4.986314,6.426807,7.610227,-17.582855
178,2025-07-20,-5.393574,6.732612,7.802346,-18.589493


In [160]:
net_approval.to_csv('net_approval_RV.csv')

### Auxiliary Graph: Individual Pollsters

In [161]:
approval_polls.columns.values

array(['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size',
       'population', 'approve', 'disapprove', 'approve_adj',
       'disapprove_adj', 'net', 'url', 'pred_plus_minus', 'poll_id'],
      dtype=object)

In [162]:
appr = full_approval.copy()
appr['net_adj'] = appr['approve'] - appr['disapprove']
appr = appr[~appr['pred_plus_minus'].isna()]
pollsters = appr['pollster'].value_counts().index.values
pollsters

array(['Rasmussen Reports', 'YouGov', 'Morning Consult', 'RMG Research',
       'J.L. Partners', 'Echelon Insights', 'Harris Insights & Analytics',
       'Global Strategy Group/GBAO (Navigator Research)',
       'Quinnipiac University', 'American Research Group',
       'Emerson College', 'McLaughlin & Associates',
       'John Zogby Strategies', 'Clarity Campaign Labs', 'Marist College',
       'InsiderAdvantage', 'Beacon Research/Shaw & Co. Research',
       'Marquette University Law School', 'Civiqs', 'Cygnal',
       'co/efficient', 'TIPP Insights', 'Wick', 'The Winston Group',
       'Targoz Market Research', 'Trafalgar Group', 'Ipsos',
       'Blueprint Polling', 'Fabrizio Lee & Associates/Impact Research',
       'SSRS', 'GBAO',
       'Hart Research Associates/Public Opinion Strategies',
       'North Star Opinion Research', 'Remington Research Group',
       'Public Opinion Strategies', 'American Pulse Research & Polling',
       'OH Predictive Insights', 'The New York Times/

In [163]:
def approval_today(data: pd.DataFrame):
    # date_range = pd.date_range(start=datetime.date(2025, 1, 23), end=datetime.datetime.today(), freq='d', inclusive='both')
#     yes_averages = []
#     no_averages = []
#     yes_stdevs = []
#     no_stdevs = []
    pipelined_df = avgs_pipeline(data, datetime.datetime.today())
    yes_avg = np.sum(pipelined_df['approve'] * pipelined_df['total_weights'])
    no_avg = np.sum(pipelined_df['disapprove'] * pipelined_df['total_weights'])
    yes_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['approve'] - yes_avg)**2))
    no_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['disapprove'] - no_avg)**2))
    net_avg = np.sum(pipelined_df['net'] * pipelined_df['total_weights'])
    net_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['net'] - net_avg)**2))
    
    #return pd.DataFrame({'Date':date_range, 'Approve':yes_averages, 'Disapprove':no_averages}), pd.DataFrame({'Date':date_range, 'Approve':yes_stdevs, 'Disapprove':no_stdevs}), avgs_pipeline(data, datetime.datetime.today())
    return yes_avg, no_avg, yes_std, no_std, net_avg, net_std

In [164]:
def indiv_pollster_avg(pollster: str):
    df = appr[appr['pollster'] == pollster]
    yes_avg, no_avg, yes_std, no_std, net_avg, net_std = approval_today(df)
    return yes_avg, no_avg, yes_std, no_std, net_avg, net_std

In [165]:
def get_ppm(pollster: str):
    df = ratings[ratings['Pollster'] == pollster]
    ppm = df['Predictive Plus-Minus'].values[0]
    
    return ppm

In [166]:
appr.shape

(326, 26)

In [167]:
def get_num_polls(pollster: str):
    df = appr[appr['pollster'] == pollster]
    num_polls = df.shape[0]
    return num_polls

In [168]:
get_ppm('YouGov')

0.047635015

In [169]:
indiv_pollster_avg('YouGov')

(43.680192712974126,
 53.703905654341185,
 0.9773975068598532,
 0.5987741979485178,
 -10.023712941367064,
 1.472832546328657)

In [170]:
indiv_pollster_avg('RMG Research')

(51.275175661102,
 47.42318096189484,
 1.0425589969416444,
 0.7645516361778093,
 3.8519946992071556,
 1.2497623818949046)

In [171]:
nets = []
net_stdevs = []
ppms = []
nums = []

for p in pollsters:
    _, _, _, _, net, net_std = indiv_pollster_avg(p)
    ppm = get_ppm(p)
    num_polls = get_num_polls(p)
    
    nets.append(net)
    net_stdevs.append(net_std)
    ppms.append(ppm)
    nums.append(num_polls)

In [172]:
# nets

In [173]:
indiv_avgs = pd.DataFrame({'Pollster': pollsters, 'Net Approval': nets, 'Net Stdev': net_stdevs, 'Predictive Plus-Minus': ppms, 'Number of Approval Polls Conducted': nums})
indiv_avgs = indiv_avgs.sort_values(['Net Approval'], ascending=True)
indiv_avgs

,Pollster,Net Approval,Net Stdev,Predictive Plus-Minus,Number of Approval Polls Conducted
9,American Research Group,-20.922665,0.806705,0.818141,6
29,SSRS,-13.969170,0.428984,0.966835,2
8,Quinnipiac University,-13.956826,0.605092,-0.290484,7
26,Ipsos,-12.917644,0.903755,0.442898,2
37,The New York Times/Siena College,-12.000000,0.000000,-0.763947,1
38,NewsNation,-12.000000,0.000000,0.614459,1
23,The Winston Group,-11.027067,0.327928,-0.136293,2
24,Targoz Market Research,-10.994006,0.189547,0.731655,2
1,YouGov,-10.023713,1.472833,0.047635,30
14,Marist College,-9.007378,0.093157,-0.436610,3


In [174]:
indiv_avgs.to_csv('indiv_pollster_avgs_RV.csv')

In [175]:
reg = lm.LinearRegression()
reg.fit(indiv_avgs[['Predictive Plus-Minus']], indiv_avgs['Net Approval'])
reg.coef_, reg.intercept_

(array([-1.56329993]), -5.0204371450912975)

In [176]:
r = r_regression(indiv_avgs[['Predictive Plus-Minus']], indiv_avgs['Net Approval'])
r

array([-0.13405298])